In [2]:
import requests
import json
import pandas as pd
from pandas.io.json import json_normalize
import urllib.parse

In [3]:
# 1. Requests
# Find Bara a flight home: She lives in Bristol, UK and would like to go to Prague, CZ for Christmas this year. 
# she prefers to arrives to the city within 5 days prior to 24.12. and save as much money as possible. 
# No changes in flights, please.

# Hint: you can get back to kiwi.com APIs to get your answer.


url = 'https://api.skypicker.com/flights?flyFrom=BRS&to=PRG&dateFrom=19/12/2019&dateTo=24/12/2019&partner=picky'
page = requests.get(url)
json_context = json.loads(page.text)
df = json_normalize(json_context['data'])

time_lower_bound = 1576684800
time_upper_bound = 1577116800

a = df[(df['dTime'] > time_lower_bound) & (df['dTime'] < time_upper_bound)&(df['price'] == df['price'].min()) ]
a['price']



0    81
Name: price, dtype: int64

In [4]:

api = 'https://api.skypicker.com/aggregation_flights?'
url = api + urllib.parse.urlencode({"fly_from": 'BRS',
                                    "fly_to": 'PRG',
                                    "v":3,
                                    "date_from": "19/12/2019",
                                    "date_to":"24/12/2019",
                                    "flight_type": "oneway",
                                    "one_for_city": 0,
                                    "one_per_date": 1,
                                    "adults":1,
                                    "curr":"EUR",
                                    "locale": "en",
                                    "max_stopovers": 0,
                                    "conn_on_diff_airport": 0,
                                    "sort": "price",
                                    "asc": 1})
data = requests.get(url).json()
print(url)
print(data)

https://api.skypicker.com/aggregation_flights?fly_from=BRS&fly_to=PRG&v=3&date_from=19%2F12%2F2019&date_to=24%2F12%2F2019&flight_type=oneway&one_for_city=0&one_per_date=1&adults=1&curr=EUR&locale=en&max_stopovers=0&conn_on_diff_airport=0&sort=price&asc=1
{'search_id': '4fd03bbe-9f49-4454-bb8a-5dfad5b0536e', 'data': [{'date': '2019-12-19', 'price': 91}, {'date': '2019-12-20', 'price': 99}, {'date': '2019-12-21', 'price': 136}, {'date': '2019-12-22', 'price': 103}, {'date': '2019-12-23', 'price': 96}, {'date': '2019-12-24', 'price': 87}], 'connections': [], 'time': 1, 'currency': 'EUR', 'currency_rate': 1.0, 'fx_rate': 1.0, 'refresh': [], 'del': 0, 'ref_tasks': [], 'search_params': {'flyFrom_type': 'airport', 'to_type': 'airport', 'seats': {'passengers': 1, 'adults': 1, 'children': 0, 'infants': 0}}, 'airlinesList': [], 'airportsList': [], 'all_airlines': [], 'all_prices': {'84-89': 1, '89-94': 1, '94-100': 2, '100-106': 1, '133-141': 1}, 'all_stopover_airports': [], 'best_results': [{'s

In [5]:
# 2. BeautifulSoup
# Alza.co.uk is an online store predominantly offering electronics and home appliances.
# First part:
# Alza.co.uk is an online store predominantly offering electronics and home appliances.
# Your task is to get the names of the best three laptops in each of these three categories
# - home and office, gaming, professional, their previous and current price 
# (if the type is not on sale, the price will be the same).

from bs4 import BeautifulSoup
import re

In [7]:
#gaming laptop
game_page = requests.get('https://www.alza.co.uk/gaming-laptops/18848814.htm')
soup = BeautifulSoup(game_page.content,'html.parser')
game_data = soup.find_all('div',{"class":"firstRow"})
game_data

[<div class="box browsingitem js-box centeredPrice canBuy inStockAvailability first firstRow" data-almostnew="false" data-brand="Lenovo" data-category="Metal" data-code="NT358o3u5h" data-dl-impression="1" data-dl-type="product" data-id="5251042" data-list="h1[itemprop='name']" data-name="Lenovo IdeaPad 720s-15IKB Silver Metallic" data-position="#boxes" data-position-class=".box" data-price="849" data-variant="Variant">
 <div class="top">
 <div class="gbbRating Good">
 <span class="alzaico-f-price"></span>
 <span>Classic Range</span>
 </div>
 <div class="bi js-block-image">
 <a class="pc browsinglink" href="/lenovo-ideapad-720s-15kb?dq=5251042">
 <em>
 <img alt="Lenovo IdeaPad 720s-15IKB Silver Metallic - Ultrabook" class="lazy js-box-image box-image" data-src="" data-srcset="https://cdn.alza.co.uk/Foto/f11/NT/NT358o3u5h.jpg, https://cdn.alza.co.uk/Foto/f9/NT/NT358o3u5h.jpg 2560w" height="210" id="img5251042" src="https://cdn.alza.co.uk/Foto/f11/NT/NT358o3u5h.jpg" width="210"/>
 </em>
 

In [206]:
for i in game_data:
    print(i["data-name"])
    print(f'current price : {i["data-price"]}')
    print(i.find_all('b')[0].text)   

Lenovo IdeaPad 720s-15IKB Silver Metallic
current price : 889
£740.83
Lenovo Legion Y530-15ICH Black
current price : 1,220.90
£1,017.42
Dell G7 17 (7790) Gaming Black
current price : 1,629
£1,357.50


In [207]:
#Home and office
home_page = requests.get('https://www.alza.co.uk/basic-home-and-office-laptops/18843464.htm')
soup = BeautifulSoup(home_page.content,'html.parser')
home_data = soup.find_all('div',{"class":"firstRow"})

In [208]:
for i in home_data:
    print(i["data-name"])
    print(f'current price : {i["data-price"]}')
    print(i.find_all('b')[0].text)

Microsoft Surface Go 128GB 8GB LTE
current price : 659
£549.17
Dell Latitude 5590
current price : 1,084.90
£904.08
Dell G7 17 (7790) Gaming Black
current price : 1,629
£1,357.50


In [210]:
#professional 
pro_page = requests.get('https://www.alza.co.uk/professional-laptops/18853299.htm')
soup = BeautifulSoup(pro_page.content,'html.parser')
pro_data = soup.find_all('div',{"class":"firstRow"})

In [211]:
for i in pro_data:
    print(i["data-name"])
    print(f'current price : {i["data-price"]}')
    print(i.find_all('b')[0].text)

Lenovo V330-14IKB Iron Grey
current price : 570.90
£475.75
Dell Latitude 5590
current price : 1,084.90
£904.08
Lenovo ThinkPad T480
current price : 1,929
£1,607.50


In [216]:
# Second part:

# Find another website of your choice, from which you can scrape products for comparison.
# Repeat the process from the first part.

#jb-hifi
#gaming-laptop
jbhifi_game_page = requests.get('https://www.jbhifi.com.au/computers-tablets/gaming-laptops/')
jbhifi_soup = BeautifulSoup(jbhifi_game_page.content,'html.parser')
jbhifi_game_data = jbhifi_soup.find_all('div',{"class":"span03 product-tile"})

In [299]:
for i in jbhifi_game_data[:10]:
    print(i.find_all('h4')[0].text)
    current_context = i.find_all('span',{'class':"amount regular"})[0].text
    current = re.sub('\s','',current_context)
     
    if len(current) != 0:
        print(current)
    else:
        sales = list(i.find_all('span',{'class':"amount onSale"}))
        current = sales[0].text
        current = re.sub('\s','',current)
        if len(current) != 0:
            print(current)
        else:
            sp_sales = i.find_all('span',{'class':"amount"})
            print(sp_sales)

Razer Blade 15 Advanced Model 15.6" 240Hz Full HD Gaming Laptop (512GB)
$4399
Razer Blade 15 Advanced Model 15.6" 240Hz Full HD Gaming Laptop (256GB)
$4099
Razer Blade Pro 17.3" Full HD 144Hz Gaming Laptop [RTX 2070]
$4799
Razer Blade 15 Advanced Model 15.6" 240Hz Full HD Gaming Laptop (512GB) [RTX 2080]
$5099
Razer Blade Pro 17.3" Full HD 144Hz Gaming Laptop [RTX 2060]
$4299
Razer Blade Pro 17.3" Full HD 144Hz Gaming Laptop [RTX 2080]
$5449
Razer Blade Stealth 13.3" Full HD Gaming Laptop (Graphics)
$2399
Razer Blade Stealth 13.3" Full HD Gaming Laptop (Base)
$1999
Acer Nitro 5 AN515-42-R3EE 15.6" Full HD Gaming Laptop
$1499
Alienware M17 17.3" Full HD Gaming Laptop
$3999


In [303]:
#home and office
jbhifi_home_page = requests.get("https://www.jbhifi.com.au/computers-tablets/2-in-1s/")
jbhifi_home_soup = BeautifulSoup(jbhifi_home_page.content,'html.parser')
jbhifi_home_data = jbhifi_home_soup.find_all('div',{"class":"span03 product-tile"})
#jbhifi_home_data

[<div class="span03 product-tile" title='Microsoft Surface Book 2 13.5" i5 256GB'>
 <div class="content" data-productid="904095">
 <a class="link" href="/computers-tablets/laptops/microsoft/microsoft-surface-book-2-13-5-i5-256gb/378677/">
 <div class="image">
 <img alt='Microsoft Surface Book 2 13.5" i5 256GB' data-src="/FileLibrary/ProductResources/Images/280199-M-LO.jpg" data-src-retina="/FileLibrary/ProductResources/Images/280199-M-HI.jpg" height="300" onerror="this.onerror=null;this.src='https://www.jbhifi.com.au/img/placeholder.gif';" src="https://www.jbhifi.com.au/img/placeholder.gif" width="300"/>
 </div>
 <div class="info">
 <div class="info-wrapper">
 <div class="brand">
 <img alt="MICROSOFT" data-src="https://www.jbhifi.com.au/Global/Brands/Logos-Croppedx40/Logo-Microsoft_Cropped.png" height="29" onerror="this.onerror=null;this.src='https://www.jbhifi.com.au/img/placeholder.gif';" src="https://www.jbhifi.com.au/img/placeholder.gif" width="150"/>
 </div>
 <!-- SKU: 378677 ID: 

In [306]:
for i in jbhifi_home_data[:10]:
    print(i.find_all('h4')[0].text)
    current_context = i.find_all('span',{'class':"amount regular"})[0].text
    current = re.sub('\s','',current_context)
     
    if len(current) != 0:
        print(current)
    else:
        sales = list(i.find_all('span',{'class':"amount onSale"}))
        current = sales[0].text
        current = re.sub('\s','',current)
        if len(current) != 0:
            print(current)
        else:
            sp_sales = i.find_all('span',{'class':"amount"})
            print(sp_sales)

Microsoft Surface Book 2 13.5" i5 256GB
$2197
Dell Inspiron 14 5000 14" 2-in-1 Laptop [Ryzen7]
$1899
Dell Iinspiron 14 5000 14" 2-in-1 Laptop [Ryzen5]
$1299
HP Pavilion x360 14-dh0046TU 14" 2-in-1 Touchscreen Laptop
$1699
Acer Spin 1 SP111-33-P8N8 11.6" Full HD 2-in-1 Laptop
$499
Acer Spin 3 SP314-52-392D 14" Full HD 2-in-1 Laptop (i3)
$699
Lenovo Ideapad C340 14" HD 2-in-1 Touchscreen Laptop
$1198
Acer Spin 3 SP314-52-555S 14" Full HD 2-in-1 Laptop (i5)
$999
Lenovo Ideapad C340 14" HD 2-in-1 Laptop
$799
Lenovo Yoga 530 HD 14" 2-in-1 Laptop
$899
Dell Inspiron 14-5000 2-in-1 Touchscreen Laptop (i3)
$999
Lenovo Yogabook C930-074 10.8" 2-in-1 Dual Display Laptop
$1999
HP Envy x360 15-CN0026TX 15.6" 2-in-1 Laptop
$1999
HP Pavilion x360 14-CD0113TU 14" 2-in-1 Touchscreen Laptop [Pentium]
$649
HP Spectre Folio Convertible 13-AK0012TU 13" 2-in-1 Laptop (Congac Brown) [512GB]
$3299
HP Spectre Folio Convertible 13-AK0015TU 13.3" 2-in-1 4G Laptop (Cognac Brown/Alloy) [512GB]
$3499
HP Spectre Fol